# Task 8.1: QASM Types and Structures

**Overview:**

**QASM** (Quantum Assembly Language) is a low-level programming language designed for describing quantum circuits and operations. It serves as an intermediate representation between high-level quantum programming languages (like Qiskit) and actual quantum hardware.


## Objective 1 : QASM Types and Casting

### Identifiers

* **Start with**: [A-Za-z], an underscore or an element from the Unicode character categories Lu/Ll/Lt/Lm/Lo/Nl
* **Continuation characters**: Allowed characters include the initial characters plus decimal numerals [0-9]
* **Reserved identifiers**: May not override reserved keywords or built-in functions
* **Examples**: 
    - Valid: `myQubit`, `_temp`, `θ` (Greek theta), `qubit1`, `result_2`
    - Invalid: `qubit@1`, `1qubit`, `qubit`

### Variables


* Named according to identifier rules
* Can be assigned values within a program
* Classical types can be initialized on declaration
* Declaration and initialization must be done one variable at a time

**Example: Quantum Variable Declaration**
```
// Declares 3 single qubits named q0 , q1 and q2
qubit q0;
qubit q1;
qubit q2;
```

**Example: Classical Variable Declaration and Initialization**
```
// 32-bit signed integer, uninitialized
int[32] a;
// 32-bit floating point, initialized to 5.5
float[32] b = 5.5;
// 3-bit register, uninitialized
bit[3] c;
// Boolean variable initialized to false
bool my_bool = false;
```

### Quantum Types

#### Qubits

* **Single qubit**: `qubit`
* **Qubit register**: `qubit[size]` (array of qubits)
* **Deprecated**: `qreg` is included for backwards compatability and will be removed in future versions
* **Global**: Quantum variables are always global

**Example: Qubit Register with Constant Size**
```
include "stdgates.inc";

// Declares 5-qubit register called q1
qubit[5] q1;
// Compile-time constant
const uint SIZE = 4;
uint runtime_u = 2;
// Declares 4-qubit register using constant
qubit[SIZE] q2;

// Apply X gate to first qubit of q1
x q1[0];                  
// Apply Z gate to third qubit of q2 (4-2=2)
z q2[SIZE - 2];
// Apply X gate to third qubit of q1 (index 2)
x q1[runtime_u];

```

#### Physical Qubits

* **Syntax**: Referenced by `$0`, `$1`, `$2`, etc.
* **Cannot be declared**: Physical qubits are predefined by hardware
* **Global**: Accessible throughout the program
* **Purpose**: Direct addressing of physical hardware qubits

**Example: Physical Qubit Addressing**
```
// Virtual qubit named gamma
qubit gamma;
// Virtual qubit using Greek letter (same as gamma)
qubit γ;
qubit[20] qubit_array;
// Apply CX gate between physical qubits 0 and 1 (not virtual)
CX $0, $1;
```

### Classical scalar types

#### Classical Bits and Registers

* **Single bit**: `bit`
* **Bit register**: `bit[size]`
* **Deprecated**: `creg` will be removed

**Example: Bit Operations**
```
// 20-bit register
bit[20] bit_array;
// 8-bit register initialized with bit string
bit[8] name = "00001111";
```

#### Integers

* **Signed**: `int[size]` or `int` (implementation-defined size)
* **Unsigned**: `uint[size]` or `uint`
* **Bit operations**: Require explicit width specification

**Example: Integer Operations and Casting**
```
// 32-bit unsigned integer
uint[32] my_uint = 10;
// 16-bit signed integer
int[16] my_int;
// Explicit cast from uint[32] to int[16]
my_int = int[16](my_uint);
```

#### Floating Point Numbers

* **Size specific**: `float[size]`
* **Hardware specific**: `float` (implementation-defined precision based on the target architecture)

**Example: Floating Point Values**
```
// 32-bit float initialized with pi constant
float[32] my_float = π;
// Machine-precision float
float my_machine_float = 2.3;
```

#### Void Type

* **Used as a**: Function return type for functions that don't return a value

#### Angles

* **Format**: `angle[size]` where 1 unit = 2π / 2^size
* **Arithmetic**: Uses unsigned integer arithmetic internally

**Example: Angle Representations**
```
// 20-bit angle representing π/2
angle[20] my_angle = π / 2;
// Machine-precision angle
angle my_machine_angle;

// Binary: "1000" (4-bit representation of π), 1 unit = π/8 , π = 8 units = "1000" 
angle[4] my_pi = π;
// Binary: "010000" (6-bit representation of π/2) , 1 unit = π /32 ,  π/2 =16 units = "010000" 
angle[6] my_pi_over_two = π/2;
// Binary: "01110000" (8-bit representation) , 1 unit = π/128 , 7*π/8 = 112 untis = "001110000"
angle[8] my_angle = 7 * (π / 8);
```

#### Complex Numbers

* **Format**: `complex[float[size]]`
* **Imaginary unit**: `im` suffix (e.g., `3.5im`)
* **Extraction**: `real()` and `imag()` functions
* **Hardware support**: May not be supported on real quantum hardware

**Example: Complex Number Operations**
```
// 64-bit complex number
complex[float[64]] c;
// Assign real and imaginary parts
c = 2.5 + 3.5im;
// Complex expression
complex[float] d = 2.0+sin(π/2) + (3.1 * 5.5 im);
// Extract real part: ~3.0
float d_real = real(d);
// Extract imaginary part: ~17.05
float d_imag = imag(d);
```

#### Boolean Types

* **Values**: `true` or `false`
* **Casting**: Can convert from `bit` using `bool()`

**Example: Boolean Operations**
```
bit my_bit = 0;
bool my_bool;
// Convert bit to boolean (0 → false)
my_bool = bool(my_bit);
// Direct assignment
my_bool = true;
```

### Compile Time Constants

* **Keyword**: `const`
* Required when specifying widths for types
* All scalar literals are implicitly const

**Example: Valid Constant Usage**
```
const uint SIZE = 32;
// 32-qubit register (uses constant)
qubit[SIZE] q1;
// 32-bit integer array
int[SIZE] i1;
```

**Example: Invalid Runtime Size**
```
// Runtime variable (not const)
uint runtime_size = 32;
// runtime_size is not a `const` type.
qubit[runtime_size] q2;
int[runtime_size] i2;
```

**Example: Constant Expressions**
```
// Constant float
const float[64] f1 = 2.5;
uint[8] runtime_u = 7;

// Valid: f1 is constant, i1 = 2 (truncated)
const int[8] i1 = int[8](f1);
// Valid: u1 = 4 (2 * 2.5 = 5, cast to uint = 4)
const uint u1 = 2 * uint(f1);

// Invalid: float[64] cannot be cast to bit[2]
const bit[2] b1 = bit[2](f1);
// Invalid: runtime_u is not constant
const int[16] i2 = int[16](runtime_u);
```

#### Built-in Constants

* **Mathematical**: `π` (pi), `τ` (tau = 2π), `e` (Euler's number)

#### Built-in Constant Expression Functions

* **Trigonometric**: `cos`, `sin`, `tan`, `arccos`, `arcsin`, `arctan` 
* **Rounding**: `ceiling`, `floor`
* **Exponential**: `exp`, `log`
* **Power/Mod**: `mod`, `pow`, `sqrt`
* **Bit manipulation**: `popcount` (count set bits), `rotl`, `rotr` (rotate left/right)

### Literals

* **Integers**: 
  - Decimal: `123`, `456`
  - Hex: `0x1F`, `0xFF`, `0x1_F` (underscores for readability)
  - Octal: `0o77`
  - Binary: `0b1010`, `0B1100`, `0b1_0101_1010`
  - Large values: `1_000_000` (underscores for readability)

* **Floating point**: 
  - Decimal: `3.14`, `.5`, `2.`
  - Scientific: `1e10`, `2e+1`, `2.0E-1`, `3.14e0`

* **Boolean**: `true`, `false`

* **Bit strings**: `"0101"`, `"1111_0000"` (underscores allowed)

* **Timing**: 
  - `10ns` (nanoseconds)
  - `2μs` or `2us` (microseconds)
  - `5ms` (milliseconds)
  - `1s` (seconds)
  - `100dt` (device-specific timing units)

### Arrays

* **Static size**: Size determined at compile time
* **Syntax**: `array[base_type, dimension1, dimension2, ...]`
* **Limitations**: Cannot be declared inside functions or gates
* **Maximum dimensions**: 7

**Example: Array Declaration and Access**
```
// 1D array of 5 integers
array[int[32], 5] myArray = {0, 1, 2, 3, 4};
// 3x2 2D array  
array[float[32], 3, 2] multiDim = {
    {1.1, 1.2}, 
    {2.1, 2.2}, 
    {3.1, 3.2}
};

// Access first element: 0
int[32] firstElem = myArray[0];
// Access last element: 4
int[32] lastElem = myArray[4];
// Negative indexing: also 4
int[32] alsoLastElem = myArray[-1];
// Access element at [1,0]: 2.1
float[32] row2col1 = multiDim[1, 0];
```

### Timing

#### Duration

* **Purpose**: Represent time intervals
* **Units**: `ns`, `μs`/`us`, `ms`, `s`, `dt` (device-specific)
* **Operations**: Can be added, subtracted, compared

**Example: Duration Operations**
```
duration one_second = 1000ms;                 // 1000 milliseconds = 1 second
duration thousand_cycles = 1000dt;            // Device-specific timing
duration two_seconds = one_second + 1s;       // Arithmetic: 1s + 1s = 2s
duration c = durationof({x $3;});            // Measure duration of gate sequence
// Note: dt units depend on the quantum hardware backend
```

#### Stretch

* **Subtype**: Specialized duration type
* **Resolution**: Determined at compile time
* **Usage**: For timing that scales with circuit execution

### Aliasing

* **Keyword**: `let`
* **Purpose**: Create alternative names for qubits and registers, Creates references, not copies

**Example: Register Aliasing**
```
// 5-qubit register
qubit[5] q;
// Alias for qubits q[1], q[2], q[3]
let myreg = q[1:4];
// Applies X to q[1] (through alias)
x myreg[0];
// Note: myreg[0] refers to q[1], myreg[1] refers to q[2], etc.
```

### Index Set and Slicing

#### Register Concatenation and Slicing

* **Concatenation**: `++` operator (same type registers)
* **Indexing**: Single index, comma-separated list, or range
* **Range syntax**: `a:b` (a to b-1), `a:c:b` (a to b-1 with step c)

**Example:  Register Operations**
```
// 2-qubit register
qubit[2] one;
// 10-qubit register
qubit[10] two;
// 12-qubit alias (2 + 10)
let concatenated = one ++ two;

// First qubit (one[0])
let first = concatenated[0];
// Last qubit (two[9])
let last = concatenated[-1];
// Qubits 0, 3, 5 from 'two'
let qubit_selection = two[{0, 3, 5}];

// First 7 qubits
let sliced = concatenated[0:6];
// Every second qubit (0,2,4,6,8,10)
let every_second = concatenated[0:2:12]; 

// Last 3 qubits (indices 7,8,9)
let last_three = two[-4:-1];
// Concatenate two aliases
let both = sliced ++ last_three;
```

#### Classical Bit Value Slicing

* **Applicable to**: `int`, `uint`, `angle` types
* **Syntax**: is similar to register slicing

**Example: Bit-level Operations on Integers**
```
// Decimal 15 = 0xF = 0b0000...1111
int[32] myInt = 15;
// Least significant bit: 1
bit[1] lastBit = myInt[0];
// Most significant bit (sign): 0
bit[1] signBit = myInt[31];
// Most significant bit also using negative index: 0
bit[1] alsoSignBit = myInt[-1];   

// Every second bit starting from 0: bits 0,2,4,...,30
bit[16] evenBits = myInt[0:2:31];
// Bits 16 through 30 (inclusive)
bit[16] upperBits = myInt[-16:-1];
// Bits 31 down to 16 (reverse order)
bit[16] upperReversed = myInt[-1:-16];

// Set bits 4-7 to binary 1010
myInt[4:7] = "1010";
// Result: myInt becomes 0xAF (decimal 175)
```

**Example: Array Element Bit Access**
```
array[int[32], 5] intArr = {0, 1, 2, 3, 4};
// Set bit 0 of element 0 to 1
intArr[0][0] = 1;
// Result: intArr[0] becomes 1 (was 0)

// Copy lowest 5 bits of intArr[4] to b
bit[5] b = intArr[4][0:4];
// intArr[4] = 4 = 0b100, so b = "00100" (5 bits)
```

#### Array Concatenation and Slicing

* **Concatenation**: `++` operator (creates new array, copies data)
* **Slicing**: Range syntax for extracting sub-arrays
* **Limitation**: Cannot pass concatenated arrays directly to subroutines

**Example: Array Operations**
```
array[int[8], 2] first = {0, 1};
array[int[8], 3] second = {2, 3, 4};

// {0, 1, 2, 3, 4}
array[int[8], 5] concat = first ++ second;
// {0, 1, 0, 1}
array[int[8], 4] selfConcat = first ++ first;

// Extract elements 1-2: {3, 4}
array[int[8], 2] secondSlice = second[1:2];
// Replace slice: second becomes {2, 0, 1}
second[1:2] = first[0:1];

array[int[8], 4] third = {5, 6, 7, 8};
// Complex assignment
selfConcat[0:3] = first[0:1] ++ third[1:2];

// subroutine_call(first ++ third)  // Not allowed: Cannot pass temporary concatenation
// subroutine_call(selfConcat)      // OK: Pass existing array variable
```

**Example: Multi-dimensional Array Operations**
```
int[8] scalar;
array[int[8], 2] oneD;
array[int[8], 3, 2] twoD;
array[int[8], 3, 2] anotherTwoD;
array[int[8], 4, 3, 2] threeD;

// Assign to single element
threeD[0, 0, 0] = scalar;
// Assign to 1D slice (first row, first column pair)
threeD[0, 0] = oneD;
// Assign to 2D slice (first 3×2 block)
threeD[0] = twoD;

// ERROR: Shape mismatch (3×2 vs 2)
threeD[0] = oneD;
// ERROR: Shape mismatch (2 vs 1)
// threeD[0, 0] = scalar

// Copy slice: rows 1-2 from anotherTwoD to twoD
twoD[1:2] = anotherTwoD[0:1];
// Copy specific column slice
twoD[1:2, 0] = anotherTwoD[0:1, 1];
```

### Casting Specifics

* **Explicit casting**: Use `type(value)` syntax for controlled conversions
* **Loss of precision**: Casting from larger to smaller types may truncate data
* **Implicit promotion**: In binary operations, the "lesser" type is cast to the "greater" type
* **Type hierarchy** (from greater to lesser):  complex > float > int > uint > bool > bit
* **Sign handling**: Casting between signed and unsigned types requires attention to sign bits

#### Allowed Casts Matrix

| From \ To | bool | int | uint | float | bit | angle |
|-----------|------|-----|------|--------|-----|-------|
| **bool**  |  -   |  ✓  |  ✓   |   ✓    |  ✓  |   ✘   |
| **int**   |  ✓   |  -  |  ✓   |   ✓    |  ✓  |   ✘   |
| **uint**  |  ✓   |  ✓  |  -   |   ✓    |  ✓  |   ✘   |
| **float** |  ✓   |  ✓  |  ✓   |   -    |  ✘  |   ✓   |
| **bit**   |  ✓   |  ✓  |  ✓   |   ✘    |  -  |   ✘   |
| **angle** |  ✓   |  ✘  |  ✘   |   ✘    |  ✓  |   -   |

**Special Cases**:
* **qubit**: Cannot be cast to or from any other types
* **duration** Cannot be cast to or from any other types except to float if the division operator is used

**Example: Type Promotion in Operations**
```
int[8] a = 5;
float[16] b = 2.5;
float[16] c = a + b;  // a is promoted to float[16] before addition
// Result: c = 7.5 (5.0 + 2.5)

uint[4] x = 3;
bit[4] y = "0101";
uint[4] z = x + uint[4](y);  // y must be explicitly cast to uint
// Result: z = 8 (3 + 5)
```

---